### PROGETTO FOUNDATION OF COMPUTER SCIENCE

#### Alex Ceccotti & Pietro De Simoni

In [1]:
import pandas as pd
import numpy as np
import ast
import re
credits = pd.read_csv("C:/Users/alexc/Documents/python_lab/movies/tmdb_5000_credits.csv")
credits.cast = credits.cast.apply(ast.literal_eval)
credits.crew = credits.crew.apply(ast.literal_eval)
movies = pd.read_csv("C:/Users/alexc/Documents/python_lab/movies/tmdb_5000_movies.csv")
movies.genres = movies.genres.apply(ast.literal_eval)
movies.keywords = movies.keywords.apply(ast.literal_eval)
movies.production_companies = movies.production_companies.apply(ast.literal_eval)
movies.production_countries = movies.production_countries.apply(ast.literal_eval)
movies.spoken_languages = movies.spoken_languages.apply(ast.literal_eval)

1) For each movie, compute the number of cast members.

In [2]:
def cast_members(movie):    
    return len(movie.cast)

In [3]:
credits["num_cast"] = credits.apply(cast_members, axis=1)
credits[["title","num_cast"]].head()

,title,num_cast
0,Avatar,83
1,Pirates of the Caribbean: At World's End,34
2,Spectre,83
3,The Dark Knight Rises,158
4,John Carter,27


2) How many movies do not have a homepage?

In [4]:
movies.homepage.isnull().sum()

3091

3) For each year, how many movies do not have a homepage?

In [5]:
movies2 = movies[(movies.release_date.notnull())&(movies.homepage.isnull())]
movies2 = movies2.reset_index()
del movies2['index']

In [6]:
def estrai_anno(data):
    assert re.search('\d\d\d\d\-\d\d\-\d\d', data)
    return(data[0:4]) 

In [7]:
movies2['year'] = movies2.release_date.apply(estrai_anno)
movies2.groupby('year').size().head()

year
1916    1
1925    1
1927    1
1929    2
1930    1
dtype: int64

4) Extract the domain of each homepage.

In [8]:
movies3 = movies[movies.homepage.notnull()]
movies3 = movies3.reset_index()
del movies3['index']

In [9]:
def estrai_dominio(url):
    assert re.search('^https*://', url)
    website = re.split('[/ ]+', url)[1]
    assert re.search('\.[a-z]+/*$', website)
    return(re.search('\.[a-z]+/*$', website).group(0))

In [10]:
movies3['dominio'] = movies3.homepage.apply(estrai_dominio)
#movies3.groupby('dominio').size().head()
movies3[["title", "homepage", "dominio"]].head()

,title,homepage,dominio
0,Avatar,http://www.avatarmovie.com/,.com
1,Pirates of the Caribbean: At World's End,http://disney.go.com/disneypictures/pirates/,.com
2,Spectre,http://www.sonypictures.com/movies/spectre/,.com
3,The Dark Knight Rises,http://www.thedarkknightrises.com/,.com
4,John Carter,http://movies.disney.com/john-carter,.com


In [11]:
def multi_homepage(url):
    urls = re.split(' ', url)
    if len(urls) == 1:
        return(False)
    else:
        return(True)
multi = movies3.homepage.apply(multi_homepage)
movies3[multi]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,dominio
1403,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",http://www.cargoderfilm.ch http://cargothemovi...,34069,"[{'id': 3388, 'name': 'space colony'}, {'id': ...",de,Cargo,The story of CARGO takes place on rusty space-...,10.142218,"[{'name': 'Telepool', 'id': 823}, {'name': 'At...",...,2009-09-24,0,120.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Cargo,5.9,140,.ch


5) Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

In [12]:
from pandas.io.json import json_normalize

def normalize_movies(colonna):
    a = colonna.apply(json_normalize)
    i = 0
    for el in movies.id:
        a[i]["movie_id"] = el
        i+=1
    return(pd.concat(a.values).reset_index().drop("index", axis=1))

def normalize_credits(colonna):
    a = colonna.apply(json_normalize)
    i = 0
    for el in credits.movie_id:
        a[i]["movie_id"] = el
        i+=1
    return(pd.concat(a.values).reset_index().drop("index", axis=1))

In [13]:
movies_keywords = normalize_movies(movies.keywords)
keywords = movies_keywords[["id", "name"]].drop_duplicates().set_index("id")
movies_keywords = movies_keywords[["id","movie_id"]]
movies_keywords.columns = ["keyword_id", "movie_id"]

In [14]:
movies_genres = normalize_movies(movies.genres)
genres = movies_genres[["id", "name"]].drop_duplicates().set_index("id")
movies_genres = movies_genres[["id","movie_id"]]
movies_genres.columns = ["genre_id", "movie_id"]

In [15]:
movies_production_companies = normalize_movies(movies.production_companies)
production_companies = movies_production_companies[["id", "name"]].drop_duplicates().set_index("id")
movies_production_companies = movies_production_companies[["id","movie_id"]]
movies_production_companies.columns = ["production_company_id", "movie_id"]

In [16]:
movies_production_countries = normalize_movies(movies.production_countries)
movies_production_countries.columns = ["id", "movie_id", "name"]
production_countries = movies_production_countries[["id", "name"]].drop_duplicates().set_index("id")
movies_production_countries = movies_production_countries[["id","movie_id"]]
movies_production_countries.columns = ["production_country_id", "movie_id"]

In [17]:
movies_spoken_languages = normalize_movies(movies.spoken_languages)
movies_spoken_languages.columns = ["id", "movie_id", "name"]
spoken_languages = movies_spoken_languages[["id", "name"]].drop_duplicates().set_index("id")
movies_spoken_languages = movies_spoken_languages[["id","movie_id"]]
movies_spoken_languages.columns = ["spoken_language_id", "movie_id"]

In [18]:
cast_norm = normalize_credits(credits.cast)
credits_cast = cast_norm[["character","credit_id","id","movie_id","order"]].set_index("credit_id")
credits_cast.columns = ["character","person_id","movie_id","order"]
people = cast_norm[["id","name","gender"]]

In [19]:
crew_norm = normalize_credits(credits.crew)
credits_crew = crew_norm[["department","credit_id","id","movie_id","job"]].set_index("credit_id")
credits_crew.columns = ["department","person_id","movie_id","job"]
people2 = crew_norm[["id","name","gender"]]
people = pd.concat([people,people2]).drop_duplicates().set_index("id")

In [20]:
movies_norm = movies[["budget", "homepage", "id", "original_language", "original_title", "overview", "popularity", "release_date", "revenue", "runtime", "status", "tagline", "title", "vote_average", "vote_count"]].set_index("id")

6) For each movie, compute the gross margin (difference between revenue and budget).

In [21]:
def compute_margin(film):
    assert (film.budget != None)|(film.revenue != None)
    return (film.revenue - film.budget)

In [22]:
movies['gross_margin'] = movies.apply(compute_margin, axis=1)
movies[['title', 'revenue', 'budget', 'gross_margin']].head()

,title,revenue,budget,gross_margin
0,Avatar,2787965087,237000000,2550965087
1,Pirates of the Caribbean: At World's End,961000000,300000000,661000000
2,Spectre,880674609,245000000,635674609
3,The Dark Knight Rises,1084939099,250000000,834939099
4,John Carter,284139100,260000000,24139100


7) For each movie, compute the number of crew members.

In [23]:
crew2 = credits_crew[["person_id", "movie_id"]].drop_duplicates()
crew_count = pd.DataFrame(crew2.groupby("movie_id").size()).reset_index()
crew_count.columns = ["movie_id", "crew_count"]
crew_count = pd.merge(crew_count, movies, left_on="movie_id", right_on="id")
crew_count[["title", "crew_count"]].head()

,title,crew_count
0,Four Rooms,78
1,Star Wars,18
2,Finding Nemo,99
3,Forrest Gump,92
4,American Beauty,107


8) For each movie, compute the number of directors.

In [24]:
crew3 = credits_crew[credits_crew.job == "Director"]
directors_count = crew3.groupby("movie_id").size().reset_index()
directors_count.columns = ["movie_id", "directors_count"]
directors_count = pd.merge(directors_count, movies, left_on="movie_id", right_on="id")
directors_count[["title", "directors_count"]].sort_values("directors_count", ascending=0).head()

,title,directors_count
701,"Paris, je t'aime",21
3918,Movie 43,12
336,Fantasia,12
3477,Fantasia 2000,8
798,Bambi,7


9) For each language, compute the number of movies where such language is spoken.

In [25]:
languages_count = movies_spoken_languages.groupby("spoken_language_id").size().reset_index()
languages_count.columns = ["languages_id", "movies_count"]
languages_count = pd.merge(languages_count, spoken_languages.reset_index(), left_on="languages_id", right_on="id")
languages_count[["languages_id", "name", "movies_count"]].sort_values("movies_count", ascending=0).head()

,languages_id,name,movies_count
19,en,English,4485
25,fr,Français,437
21,es,Español,351
16,de,Deutsch,262
36,it,Italiano,188


10) For each company and each decade, compute the overall revenue.

In [26]:
movies4 = movies[(movies.release_date.notnull())]
movies4 = movies4.reset_index()
del movies4['index']

In [27]:
def estrai_decade(data):
    assert re.search('\d\d\d\d\-\d\d\-\d\d', data)
    return(data[0:3]+'0')

In [28]:
movies4['decade'] = movies4.release_date.apply(estrai_decade)

In [29]:
movies5 = movies4[["id","decade","revenue"]]
companies2 = pd.merge(movies_production_companies, production_companies.reset_index(), left_on="production_company_id", right_on="id")
max_revenue = pd.merge(companies2, movies5, left_on="movie_id", right_on="id")
max_revenue = max_revenue.groupby(["production_company_id", "name","decade"]).sum().revenue
max_revenue.head()

production_company_id  name       decade
1.0                    Lucasfilm  1970       915398007
                                  1980      2430621397
                                  1990       924317558
                                  2000      2286034361
                                  2010        50365377
Name: revenue, dtype: int64

11) For each decade, compute the company with maximum revenue.

In [30]:
max_revenue.groupby("decade").idxmax()

decade
1910          (1307.0, Triangle Film Corporation, 1910)
1920          (8411.0, Metro-Goldwyn-Mayer (MGM), 1920)
1930    (1553.0, Selznick International Pictures, 1930)
1940            (3166.0, Walt Disney Productions, 1940)
1950          (8411.0, Metro-Goldwyn-Mayer (MGM), 1950)
1960                    (7576.0, Eon Productions, 1960)
1970                       (60.0, United Artists, 1970)
1980                    (4.0, Paramount Pictures, 1980)
1990                    (4.0, Paramount Pictures, 1990)
2000                       (6194.0, Warner Bros., 2000)
2010                   (33.0, Universal Pictures, 2010)
Name: revenue, dtype: object

In [31]:
max_revenue2 = pd.DataFrame(max_revenue).reset_index()
max_revenue3 = pd.DataFrame(max_revenue.groupby("decade").max()).reset_index()
pd.merge(max_revenue2, max_revenue3, on=["decade","revenue"])[["name","decade","revenue"]].sort_values("decade")

,name,decade,revenue
4,Triangle Film Corporation,1910,8394751
5,Wark Producing Corp.,1910,8394751
11,Metro-Goldwyn-Mayer (MGM),1920,26358000
6,Selznick International Pictures,1930,400176459
7,Metro-Goldwyn-Mayer (MGM),1930,400176459
8,Walt Disney Productions,1940,351747150
12,Metro-Goldwyn-Mayer (MGM),1950,73770324
10,Eon Productions,1960,598134765
3,United Artists,1970,1392271577
0,Paramount Pictures,1980,4431144486


12) In each year, how many movies have revenue smaller than the budget?

In [32]:
movies6 = movies4[movies4.gross_margin<0]
movies6 = movies6.reset_index()
del movies6['index']
movies6["year"] = movies6.release_date.apply(estrai_anno)

In [33]:
movies6.groupby("year").size().tail()

year
2012    52
2013    62
2014    59
2015    67
2016    26
dtype: int64

13) Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue

In [34]:
film_rev = movies[["id","revenue"]]
film_rev = film_rev[film_rev.revenue != 0]

In [35]:
def revenue_parts(film):
    id_film = film.id
    revenue = film.revenue
    num_act = len(credits_cast[credits_cast.movie_id==id_film])
    if num_act != 0:
        parte_revenue = revenue / ((2**num_act)-1)
        return(parte_revenue)
    else:
        return(-999)
film_rev["parte_revenue"] = film_rev.apply(revenue_parts, axis=1)

In [36]:
film_rev = film_rev[film_rev.parte_revenue != -999]

In [37]:
credits_cast2 = credits_cast.sort_values("order", ascending=0)

In [38]:
def distribuisci(film):
    lista = []
    id_film = film.id
    parte = film.parte_revenue
    i = 1
    for el in credits_cast2[credits_cast2.movie_id==id_film].iterrows():
        diz = {}
        diz["parte"] = i*parte
        diz["credit_id"] = el[0]
        lista.append(diz)
        i*=2
    return(lista)

In [39]:
film_rev["cast"] = film_rev.apply(distribuisci, axis=1)

In [40]:
def normalize_film(colonna):
    a = colonna.apply(json_normalize)
    return(pd.concat(a.values).reset_index().drop("index", axis=1))
cast_rev = normalize_film(film_rev.cast)

In [41]:
credits_cast = pd.merge(credits_cast.reset_index(), cast_rev, on="credit_id", how="outer")

In [42]:
int(round(sum(credits_cast[credits_cast.movie_id==19995].parte)))

2787965087

In [43]:
movies.revenue[0]

2787965087

14) For each actor find the total revenue attributed to him/her

In [44]:
credits_cast[credits_cast.parte.isnull()] = 0
actor_revenue = pd.DataFrame(credits_cast.groupby("person_id").sum().parte).reset_index()
actor_revenue = pd.merge(actor_revenue, people.reset_index(), left_on="person_id", right_on="id")
actor_revenue = actor_revenue[["id", "name", "parte"]]
actor_revenue.columns = ["id", "name", "total_revenue"]
actor_revenue.head().round(2)

,id,name,total_revenue
0,1.0,George Lucas,5.487000e+01
1,2.0,Mark Hamill,9.432493e+08
2,3.0,Harrison Ford,2.509048e+09
3,4.0,Carrie Fisher,2.483645e+08
4,5.0,Peter Cushing,4.846238e+07


15) Find the actor that is responsible for the most overall revenue

In [45]:
actor_revenue.sort_values("total_revenue", ascending=0).head(1)

,id,name,total_revenue
211,500.0,Tom Cruise,3.976120e+09
